In [1]:
# %pylab --no-import-all # bug tkinter
%matplotlib inline
import seaborn as sns
import sys
import pandas as pd
from ipywidgets import widgets
from IPython.display import Markdown, display, clear_output, Audio
import os
from os.path import join, isdir
from glob import iglob
import json
from pprint import pformat
import utils
import pickle
import io
import base64
from scipy.io import wavfile
import birdsonganalysis as bsa

sys.path.append('../model')

from song_model import SongModel

sns.set_palette('colorblind')

In [2]:
gridpath = '../model/res/cluster_simulation_180523_184444/'
run_paths = sorted([run_path for run_path in iglob(join(gridpath, '*')) if isdir(run_path)])

gridanalysis = utils.GridAnalyser(run_paths, figsize=(10, 10/2.3), save_fig=False)

In [ ]:
# with open(join(gridpath, 'desc.md')) as f:
#     display(Markdown(f.read()))

In [3]:
sns.set_context('talk')

vbox = widgets.VBox() # Displays multiple widgets vertically using the flexible box model

out = widgets.HTML()

tbs = []
# Organization of the toggle buttons
for options in gridanalysis.options_list:
    tb = widgets.ToggleButtons()
    tb.options = sorted(list(options))
    display(tb)
    tbs.append(tb)

def find_id(value):
    out.value = 'looking'
    v = 'Not Found'
    for i in range(len(gridanalysis.run_paths)):
        if gridanalysis.conf[i]['name'] == '+'.join([tb.value for tb in tbs]):
            slider.value = i
            v = 'Found'
            break
    out.value = v
    
slider = widgets.IntSlider(min=0, max=len(run_paths)-1)
slider.observe(lambda x: gridanalysis.show(x['new'], vbox), names='value')
display(out)
display(slider)
display(vbox)

gridanalysis.show(slider.value, vbox)

for tb in tbs:
    tb.observe(lambda x: find_id(x['new']), names='value')

ToggleButtons(options=('seed0',), value='seed0')

ToggleButtons(options=('optimise_gesture_padded', 'optimise_gesture_whole'), value='optimise_gesture_padded')

ToggleButtons(options=('gilles_fast_boari_short',), value='gilles_fast_boari_short')

ToggleButtons(options=('on_alpha_silence_beta', 'zero_alpha_zero_beta'), value='on_alpha_silence_beta')

ToggleButtons(options=('default_coef',), value='default_coef')

...

HTML(value='')

IntSlider(value=0, max=31)

VBox()

/home/gnouveau/.virtualenvs/birdsong/lib/python3.5/site-packages/seaborn/timeseries.py:183: UserWarning: The tsplot function is deprecated and will be removed or replaced (in a substantially altered version) in a future release.
  warnings.warn(msg, UserWarning)


boari score 141.02719930771806
boari score 141.02719930771806
boari score 141.02719930771806
boari score 141.02719930771806
boari score 141.02719930771806
boari score 141.02719930771806
boari score 141.02719930771806
boari score 141.02719930771806
boari score 141.02719930771806
boari score 141.02719930771806
boari score 141.02719930771806
boari score 141.02719930771806
boari score 141.02719930771806


In [4]:
#################################################################
# TEST SCORE DE PAUL SUR LES CHANTS SYNTHETISES DE BOARI
#################################################################

import matplotlib.pyplot as plt
from os.path import basename
import numpy as np
from measures import bsa_measure

liste_nom_chanson = ['ba_example', 'ba_syllable_a',
                     'ba_syllable_a_end', 'ba_syllable_b', 
                     'bells', 'bells_short', 'boari_short',
                     'flashcam', 'samba', 'simple']

liste_coefs = [{"fm": 1,
                "am": 1,
                "entropy": 1,
                "goodness": 1,
                "amplitude": 1,
                "pitch": 1},
                {"fm": 1,
                "am": 1,
                "entropy": 5,
                "goodness": 1,
                "amplitude": 5,
                "pitch": 1}]
for nom_chanson in liste_nom_chanson:
    sr, synth = wavfile.read('../data/{}_out.wav'.format(nom_chanson))                                 
    
    res1 = '{}:'.format(nom_chanson)
    res2 = '{} sans crop:'.format(nom_chanson)
    res3 = '{} sans threshold'.format(nom_chanson)
    for coefs in liste_coefs:
        gridanalysis.conf[0]['coefs'] = coefs
        amp = bsa.song_amplitude(synth, 256, 40, 1024)
    #     plt.figure(figsize=(10, 5))
    #     plt.plot(amp)
    #     plt.title('amp')
    #     plt.show()

        sort_amp = np.sort(amp)
        i_max_diff = np.argmax(utils._running_mean(np.diff(sort_amp), 100))
        th = sort_amp[i_max_diff]

        sort_amp = sort_amp[len(sort_amp)//10:]
    #     plt.figure(figsize=(10, 5))
    #     plt.plot(sort_amp)
    #     plt.title('sort_amp')
    #     plt.show()

    #     amp_diff = np.diff(sort_amp)
    #     plt.figure(figsize=(10, 5))
    #     plt.plot(amp_diff)
    #     plt.title('amp_diff')
    #     plt.show()

    #     amp_diff_smoothed = utils._running_mean(np.diff(sort_amp), 100)
    #     plt.figure(figsize=(10, 5))
    #     plt.plot(amp_diff_smoothed)
    #     plt.title('amp_diff_smoothed')
    #     plt.show()

        i_max_diff = np.argmax(utils._running_mean(np.diff(sort_amp), 100))
        threshold = sort_amp[i_max_diff]

    #     plt.figure(figsize=(10, 5))
    #     plt.plot(amp)
    #     plt.axhline(y=threshold, color="k", linewidth=1)
    #     plt.title('amp')
    #     plt.show()

        sr, tutor = wavfile.read('../data/{}.wav'.format(nom_chanson))
        msynth = bsa_measure(synth, 44100, coefs=gridanalysis.conf[0]['coefs'])
        mtutor = bsa_measure(tutor, 44100, coefs=gridanalysis.conf[0]['coefs'])
        score = np.linalg.norm(msynth[amp > threshold] - mtutor[amp > threshold]) / np.sum(amp > threshold) * len(amp)

        res1 += '\t' + str(score)

        score = np.linalg.norm(msynth[amp > th] - mtutor[amp > th]) / np.sum(amp > th) * len(amp)
        res2 += '\t' + str(score)

        score = np.linalg.norm(msynth - mtutor)

        res3 += '\t' + str(score)
    
    print(res1)
    print(res2)
    print(res3)
    print()

ba_example:	264.09166641333206	437.1641643289677
ba_example sans crop:	264.09166641333206	437.1641643289677
ba_example sans threshold	264.8422811350109	728.8412619379059

ba_syllable_a:	49.11580468593274	72.22340563047308
ba_syllable_a sans crop:	48.68630985550633	68.36210072116495
ba_syllable_a sans threshold	48.912692603418726	68.11400777443828

ba_syllable_a_end:	41.834222001162956	60.35648897773306
ba_syllable_a_end sans crop:	40.44343214560718	56.744799510162125
ba_syllable_a_end sans threshold	40.147448123418684	55.803978590344094

ba_syllable_b:	79.74960927481429	119.66241622830918
ba_syllable_b sans crop:	74.23203947543155	111.8401642664906
ba_syllable_b sans threshold	73.65774030773505	110.95777458935962

bells:	1848.6922714211405	2761.6091343714756
bells sans crop:	339.8897523505728	681.4822216300065
bells sans threshold	331.0653445041053	718.7729354082376

bells_short:	260.32592217437065	471.91994565134684
bells_short sans crop:	242.67019989855606	445.9723113771407
bells_sho